In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

# Data Cleaning

Przed uruchomieniem naszych danych przez model ważnym krokiem jest **Eksploracyjna analiza danych** lub EDA. EDA, jak sama nazwa wskazuje, to dogłębna analiza naszych danych. Proces EDA przeplata czyszczenie danych, radzenie sobie z brakującymi wartościami oraz wizualizację danych i ich właściwości statystycznych. Zazwyczaj wszystkie te procesy są wykonywane razem, ale dla potrzeb edukacyjnych rozbijemy je na osobne części.

### Prerequisites
[flights.txt](https://drive.google.com/file/d/1cVV3TZcxS31fk9JrskaRP1pbOfaoNcwe/view?usp=sharing)
(źródło: https://www.kaggle.com/mmetter/flights/data).
W większości przypadków dane posiadają dokumentację. **Czytanie dokumentacji danych jest ważne**!!!! Ta konkretna część danych nie ma jednak żadnej dokumentacji - będziemy musieli skorzystać z naszej intuicji oraz nazw kolumn (zmiennych).

### Typy danych

<table >
	<tbody>
		<tr>
            <td><b>Typ danych</b></td>
            <td><b>Typ danych w Pythonie</b></td>
            <td><b>Przykłady</b></td>
		</tr>
		<tr>
			<td>Dane tekstowe</td>
            <td>str</td>
			<td>Nazwiska, adresy</td>
		</tr>
		<tr>
			<td>Integers</td>
            <td>int</td>
			<td># przedmioty, # osoby</td>
		</tr>
		<tr>
			<td>Floats/Ułamki</td>
            <td>float</td>
			<td>Waluty, odległości</td>
		</tr>
		<tr>
			<td>Binary/Boolean</td>
            <td>bool</td>
			<td>Pytania zamknięte, yes/no</td>
		</tr>
		<tr>
			<td>Data (i czas)</td>
            <td>datetime</td>
			<td>Data wysyłki, czas przyjazdu</td>
		</tr>
		<tr>
			<td>Kategorie</td>
            <td>category</td>
			<td>Stany, kolory, płeć</td>
		</tr>
	</tbody>
</table>

In [4]:
path='sciezka na twoim dysku/komputerze'

In [5]:
import pandas as pd
pd.set_option('display.max_columns', None)
flights_df = pd.read_csv("C:/Users/grzeg/PycharmProjects/PAD/Cwiczenia_5/flights.txt", sep="|")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/grzeg/PycharmProjects/PAD/Cwiczenia_5/flights.txt'

In [ ]:
flights_df.head()

Zestaw danych flights jest problematyzny z wielu względów, ale zanim na nie odpowiemy, przyjrzymy się nazwom zmiennych.

- **TRANSACTIONID**: Unikalny identyfikator

- **FLIGHTDATE**: Data lotu. Wygląda na to, że jest zakodowana jako liczba zamiast obiektu daty

- **TAILNUM**: Wygląda na to, że w niektórych wierszach zawiera @@

- **ORIGAIRPORTNAME** oraz **DESTAIRPORTNAME**: Wygląda na to, że nazwa miasta i stan są połączone i dołączone przed rzeczywistą nazwą lotniska

- **CRSDEPTIME** oraz **DEPTIME**: Wygląda na to, że zawierają niepoprawnie sformatowane czasy. Wydaje się też, że: **CRSDEPTTIME** + **DEPDELAY** = **DEPTIME**

- **DEPDELAY**: Opóźnienie wyjazdu w minutach?

- **TAXIOUT**: Ile czasu upłynęło od wystartowania silnika do oderwania się od ziemi? Poza tym: **DEPTIME** + **TAXIOUT** = **WHEELSOFF**

- **WHEELSOFF**: Czas, w którym koła oderwały się od ziemi

- **WHEELSON**: Czas, w którym koła dotknęły ziemi podczas lądowania

- **TAXIIN**: Wygląda na liczbę minut od zetknięcia kół z ziemią do „parkowania”

- **CRSARRTIME**: Oczekiwany czas przylotu w formacie 24h

- **ARRTIME**: Faktyczny czas przylotu

- **ARRDELAY**: Różnica pomiędzy **CRSARRTIME** oraz **ARRTIME**

- **CRSELAPSEDTIME**: Planowany czas podróży (minuty)

- **ACTUALELAPSEDTIME**: Rzeczywisty czas podróży (minuty)

- **CANCELLED**: Czy lot został odwołany czy nie. Niektóre wartości logiczne zostały przedstawione jako False, inne jako 0. Podobnie jest z wartościami True i 1.

- **DIVERTED**: Czy samolot został przekierowany. Podobne problemy dotyczące True/False jak powyżej?

- **DISTANCE**: Odległość (całkowita) przebyta przez samolot, zakodowana jako ciąg znaków z połączonymi z nim „milami”

In [ ]:
flights_df

Tak więc przechodzenie przez każdą kolumnę identyfikuje problemy, które może mieć zestaw danych. Najpierw zajmijmy się kwestią **odległości**. Wygląda na to, że ma to być zakodowane _całkowite_ (zauważ, że na początku tego wykładu powiedziałem, że odległości mogą być zmiennoprzecinkowe. Dlaczego zmiana zdania?) i mają dopisane 'mile' na końcu numer. Możemy zrobić o wiele więcej rzeczy z danymi liczbowymi niż danymi tekstowymi, które reprezentują liczby, więc najpierw przekonwertujmy to na int.
Aalizująć tabelę kolumna po kolumnie dostrzegamy różne problemy, z jakimi musimy się zmierzyć w tym zbiorze danych. Można zacząć od problemu odległości. Wygląda na to, że odległość miała być zakodowana jako integery, ale pojawiają oznaczenia miar. Dane numeryczne pozwalają na o wiele więcej operacji niż dane tekstowe, dlatego należy wyczyścić te wartości tekstowe.

<table>
    <tr>
        <td> </td>
        <td><b>OrderID</b></td>
        <td><b>Cost</b></td>
        <td><b>Quantity</b></td>
        <td><b>Address</b></td>
    </tr>
    <tr>
        <td>0</td>
        <td>1234</td>
        <td>£1000.00</td>
        <td>10</td>
        <td>123 Fake Street</td>
    </tr>
    <tr>
        <td>1</td>
        <td>7890</td>
        <td>£35.50</td>
        <td>3</td>
        <td>789 Real Road</td>
    </tr>
    
</table>
W powyższej tabeli widzimy, że koszt powinien być liczbą zmiennoprzecinkową - jednak ma dołączony symbol £. Aby użyć tej kolumny jako float, musimy usunąć £. Jednak zanim to zrobimy, spójrzmy na typy danych kolumn. Odbywa się to poprzez wywołanie atrybutu `.dtypes` w naszej ramce danych. W powyższym przykładzie zwrócilibyśmy:


<table>
    <tr>
        <td>OrderID</td>
        <td>int64</td>
    </tr>
    <tr>
        <td>Cost</td>
        <td>object</td>
    </tr>
    <tr>
        <td>Quantity</td>
        <td>int64</td>
    </tr>
    <tr>
        <td>Address</td>
        <td>object</td>
    </tr>
</table>

Możemy również użyć metody `.info()`, która zwraca nam również informacje o wartościach null w każdej kolumnie (wkrótce omówimy, jak radzić sobie z wartościami null/brakującymi wartościami)

In [ ]:
## Znajdź typy obiektów dla każdej kolumny za pomocą .dtypes
flights_df.dtypes

In [ ]:
## Znajdź typy obiektów i liczbę wartości null za pomocą .info()
flights_df.info()


Gdybyśmy mieli zsumować powyższą kolumnę <b>cost</b> (`sales['cost'].sum()`), zwrócone zostałoby coś podobnego do następującego ciągu:
```£1000,00£35.50£46.10£76.35```...

Oczywiście nie o to nam chodzi... wolimy zsumować wszystkie nasze koszty.

Powtórz ten krok z kolumną „DISTANCE” z danymi dotyczącymi lotów

In [ ]:
## Zsumuj pierwsze 10 wystąpień kolumny „DISTANCE” w danych lotów.
# Bądź świadomy tego, gdzie używasz slicingu :). Jaka jest techniczna różnica między slicingiem przed .sum() i po?
flights_df['DISTANCE'][:10].sum()

Aby rozwiązać problem z naszymi danymi sprzedaży, musimy zrobić dwie rzeczy:
1. Usuąć „£”
2. Zamienić kolumnę na zmiennoprzecinkowy typ danych

Odbywałoby się to w następujący sposób:
```python
sales['cost'] = sales['cost'].str.strip('£')
sales['cost'] = sales['cost'].astype('float64')
```

Uzbrojeni w tę wiedzę, zamieńmy kolumnę odległości na int!

In [ ]:
## Usuń „mile” z ramki danych
flights_df["DISTANCE"] = flights_df["DISTANCE"].str.strip(" miles")
## Zamień kolumnę na typ int64
flights_df["DISTANCE"] = flights_df["DISTANCE"].astype("int64")
## Sprawdź, czy kolumna została pomyślnie przekonwertowana na int

Świetnie! W ten sposób możemy przekonwertować nieuporządkowane dane tekstowe na liczby. Przyjrzyjmy się teraz konwersji danych na wartości kategoryczne.

W naszym zbiorze danych mamy wiele kolumn, które mogą być kategoryczne. Czy potrafisz określić, które to są?
<br>
<details>
    <summary><b>></b> Categorical variables (click to reveal)</summary>
    <ul>
        <li>AIRLINECODE</li>
        <li>AIRLINENAME</li>
        <li>ORIGINAIRPORTCODE</li>
        <li>ORIGAIRPORTNAME</li>
        <li>ORIGINCITYNAME</li>
        <li>ORIGINSTATE</li>
        <li>ORIGINSTATENAME</li>
        <li>DESTAIRPORTCODE</li>
        <li>DESTAIRPORTNAME</li>
        <li>DESTCITYNAME</li>
        <li>DESTSTATE</li>
        <li>DESTSTATENAME</li>
    </ul>
</details>

Używając metody `.describe()` możemy dostać więcej informacji o konkretnej kolumnie. Użyjmy jako przykładu `AIRLINECODE`

In [ ]:
flights_df['DISTANCE'].describe()

In [ ]:
flights_df['AIRLINECODE'].describe()

Gdy uruchomimy `.describe()` nad tą zmienną, otrzymujemy kilka (średnio) użytecznych statystyk. Widzimy jednak, że typ danych tej kolumny został zinterpretowany jako `object`. Z tabeli typów danych, którą wprowadziliśmy wcześniej, widzimy, że istnieje obsługa kategorii. Przekształćmy to w kategorię i zobaczmy różnicę w stosunku do metody opisu.

In [ ]:
flights_df['AIRLINECODE'] = flights_df['AIRLINECODE'].astype('category')
flights_df['AIRLINECODE']

In [6]:
flights_df['AIRLINECODE'].describe()

NameError: name 'flights_df' is not defined

Tak naprawdę brak widocznej różnicy po użyciu metody `.describe()` (co dziwne, wciąż zwraca dtype object)! Użyjemy metody `.info()`, aby sprawdzić nasze zużycie pamięci

In [ ]:
flights_df['AIRLINECODE'] = flights_df['AIRLINECODE'].astype('object')
flights_df.info()

In [ ]:
flights_df['AIRLINECODE'] = flights_df['AIRLINECODE'].astype('category')
flights_df.info()

Widzimy, że nasze zużycie pamięci spadło o około 8 MB po przekształceniu tej jednej kolumny w kategorię! Ok, tak, wprawdzie nie jest to wielka sprawa przy pracy z danymi o tym rozmiarze, ale pamiętaj, że oszczędność pamięci pochodzi tylko z jednej z wielu kategorycznych kolumn, które mamy.

Więc dlaczego tak jest? Cóż, pod maską Pandas reprezentują kategorie jako typy całkowite. W rzeczywistości coś, na co możesz natknąć się podczas pracy z innymi zestawami danych, to jawne wyświetlanie kolumny kategorii zakodowanej jako liczby całkowite. Zmodyfikujmy naszą ramkę danych, aby zobaczyć, co się stanie dalej.

In [ ]:
flights_df['AIRLINECODE_ASINT'] = flights_df['AIRLINECODE'].cat.codes.astype('int8')
flights_df['AIRLINECODE_ASINT']

Kiedy uruchamiamy `.describe()` możemy zobaczyć zwrócone statystyki, które nie mają sensu dla naszej kolumny:

In [ ]:
flights_df['AIRLINECODE_ASINT'].describe()

Nie ma sensu, aby kolumna kategoryczna miała średnią lub jakiekolwiek inne właściwości statystyczne.

In [ ]:
flights_df = flights_df.drop('AIRLINECODE_ASINT', 1)

Jakie typy danych (numeryczne, datetime, tekst lub kategoryczne) przypisałbyś następującym danym?:

- Opis przedmiotu
- Roczny dochód
- Rozmiar odzieży
- Czas przylotu samolotu
- Urodziny
- Smaki koktajli mlecznych w McDonalds
- Pierwsza połowa kodu pocztowego
- Pełny kod pocztowy
- Czas potrzebny biegaczom na ukończenie 5K

## Zduplikowane wartości

Innym częstym problemem, z którym możemy się spotkać, są **zduplikowane wartości**. Jak sama nazwa wskazuje, dzieje się tak, gdy te same wartości powtarzają się w wielu wierszach lub kolumnach:
<table>
    <tr>
        <td><b>first_name</b></td>
        <td><b>last_name</b></td>
        <td><b>address</b></td>
        <td><b>age</b></td>
        <td><b>income</b></td>
    </tr>
    <tr>
        <td>John</td>
        <td>Doe</td>
        <td>123 Real Street</td>
        <td>25</td>
        <td>£28000</td>
    </tr>
    <tr>
        <td>Jane</td>
        <td>Smith</td>
        <td>789 Fake Road</td>
        <td>29</td>
        <td>£32000</td>
    </tr>
    <tr>
        <td>Jane</td>
        <td>Smith</td>
        <td>789 Fake Road</td>
        <td>29</td>
        <td>£32000</td>
    </tr>
    <tr>
        <td>Mark</td>
        <td>Smith</td>
        <td>789 Fake Road</td>
        <td>31</td>
        <td>£32000</td>
    </tr>
</table>

W powyższym przykładzie widzimy, że Jane Smith ma dwa wpisy bezpośrednio zduplikowane. Jednak w niektórych przypadkach możemy zobaczyć bardzo podobne wpisy:

<table>
    <tr>
        <td><b>first_name</b></td>
        <td><b>last_name</b></td>
        <td><b>address</b></td>
        <td><b>age</b></td>
        <td><b>income</b></td>
    </tr>
    <tr>
        <td>John</td>
        <td>Doe</td>
        <td>123 Real Street</td>
        <td>25</td>
        <td>£28000</td>
    </tr>
    <tr>
        <td>Jane</td>
        <td>Smith</td>
        <td>789 Fake Road</td>
        <td><b>28</b></td>
        <td>£32000</td>
    </tr>
    <tr>
        <td>Jane</td>
        <td>Smith</td>
        <td>789 Fake Road</td>
        <td>29</td>
        <td>£32000</td>
    </tr>
    <tr>
        <td>Mark</td>
        <td>Smith</td>
        <td>789 Fake Road</td>
        <td>31</td>
        <td>£32000</td>
    </tr>
</table>

(Różnica wieku między obiema Jane Smith). Ten rodzaj duplikatu błędu jest najprawdopodobniej spowodowany problemem z wprowadzaniem danych lub ponownym przesłaniem dowolnego formularza złożonego przez Jane – który został wprowadzony do bazy danych bez usuwania jej starego wpisu.

Najczęściej jednak duplikaty danych wynikają z błędów/wzorców projektowych w potokach danych lub najczęściej z łączenia baz danych i konsolidacji danych z różnych zestawów danych/baz danych, które mogą zachować zduplikowane wartości.

Pandas udostępnia nam metodę [`.duplicated()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html). Użyjmy tego w naszej ramce danych, aby zobaczyć, co zwraca

In [ ]:
flights_df.duplicated()

Zauważ, że możemy użyć `.sum()` do wartości logicznych. Zasadniczo False są interpretowane jako 0, a True jako 1. Tak więc sumując ramkę danych, możemy uzyskać całkowitą liczbę zduplikowanych wartości!

In [ ]:
flights_df.duplicated().sum()

Brak duplikatów to dobry wynik, ale nie dajmy się zwieść. Przypomnijmy sobie duplikat Jane Smith powyżej. Zduplikowana metoda nie zwróciłaby wartości true, ponieważ cały wiersz nie był dokładnym duplikatem. Dlatego możemy w metodzie `.duplicated()` skorzystać z dwóch argumentów: `subset` i `keep`. Dla argumentu subset podaj listę nazw kolumn, w których chcemy sprawdzić duplikaty, a w argumencie keep podaj  1 z 3 wartości: „pierwsza”, „ostatnia” lub „Fałsz”. Z dokumentacji wiemy, że:
- `first` : Oznacz duplikaty jako True z wyjątkiem pierwszego wystąpienia.
- `last` : Zaznacz duplikaty jako True z wyjątkiem ostatniego wystąpienia.
- `False` : Oznacz wszystkie duplikaty jako True.

W wielu przypadkach wybranie podzbioru jest bardziej intuicyjne niż naukowe.

In [ ]:
## Znajdź duplikaty w ramce danych lotów w następujących kolumnach, ustaw keep = False:
 # "ORIGAIRPORTNAME", "DESTAIRPORTNAME", "AIRLINECODE", "FLIGHTDATE", "CRSDEPTIME", "DEPTIME", "ARRTIME"
 # Przypisz to zmiennej 'duplicates'
 # Czy wybrałam dobre klolumny? Postąpiłbyś inaczej?
duplicates = flights_df.duplicated(subset=["ORIGAIRPORTNAME", "DESTAIRPORTNAME", "AIRLINECODE", "FLIGHTDATE", "CRSDEPTIME", "DEPTIME", "ARRTIME"], keep=False)

# Używając df[duplikaty], zwracane są punkty danych, w których istnieją duplikaty.
## Zwróć duplikaty dla ramki danych lotów
flights_df[duplicates]

Jako drugorzędną obserwację widzimy, że „TALINUM” również przyjmuje wartość „UNKNOWN” dla brakujących wartości. Zanotujemy to, abyśmy mogli zająć się tym później.

Aby posortować naszą ramkę danych, możemy użyć metody [`.sort_values()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html). Przeczytaj dokumentację i użyj tej metody, aby posortować ramkę danych według nazwy kolumny, która Twoim zdaniem jest odpowiednia (taka, która pozwala łatwo zweryfikować, czy zwrócone wpisy są faktycznymi duplikatami).

In [ ]:
## Posortuj zduplikowane wartości według odpowiedniego indeksu
flights_df[duplicates].sort_values("FLIGHTDATE")


In [ ]:
flights_df["TAILNUM"].isna().sum()

### Radzenie sobie z duplikatami

Mamy dwie opcje:
1. Uśrednienie wartości, tam gdzie to możliwe
2. Usunięcie jednego z duplikatów (lub wielu, tak by pozostał pojedynczy wiersz)


##### Uśrednianie

Uśrednianie po zduplikowanych wartościach można tak naprawdę wykonać tylko na typach danych, które mają sens. W powyższej tabeli pierwsze dwa wpisy mają prawidłowe czasy, które możemy uśrednić. Ogólnie rzecz biorąc, sposób, w jaki uśredniamy, to grupowanie według odpowiednich kolumn (poprzez `.groupby()`) i łączenie tego z funkcją `.agg()`. W tym przypadku chcemy pogrupować wg kolumn w podzbiorze poza kolumnami, które nas interesują (np. wg. czasu). Naszym argumentem do `.agg()` jest słownik z parami klucz-wartość nazw kolumn i funkcją agregacji, którą chcemy nad nimi zastosować (np. suma, różnica, średnia itp.).

In [ ]:
summaries = {"CRSARRTIME": "mean", "ARRTIME": "mean", "ARRDELAY": "mean", "CRSELAPSEDTIME": "mean", "ACTUALELAPSEDTIME": "mean"}

grouped_duplicates = flights_df[duplicates].groupby(["FLIGHTDATE", "AIRLINECODE", "ORIGAIRPORTNAME", "DESTAIRPORTNAME"])
grouped_duplicates_min_transactionid = grouped_duplicates["TRANSACTIONID"].min().reset_index()

f_df_duplicates = pd.merge(
    grouped_duplicates_min_transactionid,
    grouped_duplicates.agg(summaries).reset_index(),
    how="inner"
).sort_values("TRANSACTIONID")

f_df_duplicates


In [ ]:
# Dlaczego w polu TRANSACTIONID jest teraz tak wiele nowych NaN?
## Jak się ich pozbyć?
f_df_duplicates = f_df_duplicates.dropna(subset=["TRANSACTIONID"])

## Ponownie zakoduj TRANSACTIONID do int64
f_df_duplicates["TRANSACTIONID"] = f_df_duplicates["TRANSACTIONID"].astype("int64")
f_df_duplicates

In [ ]:
# Metoda .update() pozwala nam aktualizować rekordy w jednej ramce danych na podstawie wartości w innej
# Potrzebny jest pewien sposób "powiązania" rekordów do nadpisania/aktualizacji, jeśli nie chcemy używać domyślnego indeksu ramki danych
## Tak więc, używając metody .set_index(), ustaw ramkom flight_df i f_df_duplicates nowy indeks na unikalny klucz indentifera, który obaj współdzielą
flights_df = flights_df.set_index("TRANSACTIONID")
f_df_duplicates = f_df_duplicates.set_index("TRANSACTIONID")

# Teraz możemy zaktualizować ramkę flight_df o nową ramkę danych
flights_df.update(f_df_duplicates)

## I na koniec możemy opcjonalnie zresetować indeks, aby uzyskać domyślne indeksowanie ramki danych
flights_df=flights_df.reset_index()

In [ ]:
flights_df.head(2)

In [ ]:
flights_df[flights_df["TRANSACTIONID"]==1974100]

##### Usuwanie duplikatów

Jeśli chodzi o usuwanie duplikatów, Pandas udostępnia nam metodę `.drop_duplicates()`, która przyjmuje trzy argumenty:
1. `subset`
2. `keep`
3. `inplace` - wartość boolean, czy chcemy nadpisać ramkę czy nie

In [ ]:
subset = ["ORIGAIRPORTNAME", "DESTAIRPORTNAME", "AIRLINECODE", "FLIGHTDATE", "CRSDEPTIME", "DEPTIME", "ARRTIME"]
## Używając inplace = True, usuń duplikaty. Zastanów się, jaką wartość powinniśmy ustawić argumentowi keep
flights_df.drop_duplicates(subset, 'first', inplace=True)

In [ ]:
flights_df[duplicates].tail()

## Dane kategoryczne

Wspomnieliśmy już o danych kategorycznych wcześniej, ale tutaj bardziej szczegółowo zdefiniujemy to pojęcie. Zmienne danych kategorycznych przyjmują swoją wartość z predefiniowanego zestawu kategorii. Widzieliśmy powyższy przykład z kodami AIRLINE.

Czy poniższe zmienne są kategoryczne?
- TAILNUM
- FLIGHTNUM
- ORIGINAIRPORTCODE
- ORIGAIRPORTNAME
- CANCELLED

A kolumny w poniższej tabeli?

<table>
    <tr>
        <td><b>First Name</b></td>
        <td><b>Last Name</b></td>
        <td><b>Age</b></td>
        <td><b>Address</b></td>
        <td><b>District Postcode</b></td>
        <td><b>Full Postcode</b></td>
        <td><b>Married</b><td>
    </tr>
    <tr>
        <td>John</td>
        <td>Doe</td>
        <td>31</td>
        <td>123 Fake Street, Realtown</td>
        <td>RT1</td>
        <td>RT1 3NV</td>
        <td>True</td>
    </tr>
    <tr>
        <td>Diane</td>
        <td>Smith</td>
        <td>31</td>
        <td>42 World Road, Realtown</td>
        <td>RT2</td>
        <td>RT2 7XU</td>
        <td>False</td>
    </tr>
    <tr>
        <td>Kate</td>
        <td>Doe</td>
        <td>29</td>
        <td>123 Fake Street, Realtown</td>
        <td>RT1</td>
        <td>RT1 3NV</td>
        <td>False</td>
    </tr>
    <tr>
        <td>Charlie</td>
        <td>Doe</td>
        <td>33</td>
        <td>789 Real Road, Fakecity</td>
        <td>FC2</td>
        <td>FC2 9ER</td>
        <td>True</td>        
    </tr>    
</table>
Dane kategoryczne mogą przyjmować tylko jedną ze skończonego zestawu wartości i nie jest możliwe, aby wykroczyły poza te uprzednio zdefiniowane kategorie. Jednak podczas procesu zbierania danych może wystąpić szum w naszych danych (np. jeśli nasze dane kategoryczne zostały zebrane za pomocą okienka tekstowego do wprowadzania dowolnej treści).


Istnieje kilka sposobów radzenia sobie z niespójnymi kategoriami:
1. Usuwanie danych
2. Zmiana mapowania kategorii
3. Wnioskowanie kategorii

### Usuwanie danych


Przyjrzyjmy się kolumnie `ORIGINSTATENAME`. Usunięcie danych jest wymagane, gdy mamy wartość, która w naszym wpisie nie znajduje się (koncepcyjnie) we wstępnie zdefiniowanym zestawie kategorii. Zaczniemy od zwrócenia wszystkich unikalnych wartości w zmiennej.

In [ ]:
## skonstruuj zbiór unikalnych wartości w ORIGINSTATENAME
states = set(flights_df["ORIGINSTATENAME"])
states

Załóżmy teraz, że otrzymaliśmy kilka nowych wpisów, których nazwy stanów nie występują w tym predefiniowanym zestawie kategorii (na przykład „Fakestate”).

In [ ]:
## używając metody .at() lub .iat() na flight_df, zmodyfikuj jeden z wierszy
## w naszej tabeli, aby mieć ORIGINSTATENAME jako Fakestate
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.at.html
flights_df.at[len(flights_df), "ORIGINSTATENAME"] = "Fakestate"

In [ ]:
# Wyświetl unikalne wpisy w ORIGINSTATENAME w naszej zmodyfikowanej ramce danych
flights_df["ORIGINSTATENAME"].unique()
# LUB set(flights_df["ORIGINSTATENAME"])

In [ ]:
## Korzystając z operacji set, znajdź różnicę między stanami początkowymi w naszej ramce danych a naszą predefiniowaną listą
inconsistent_categories = set(flights_df["ORIGINSTATENAME"]).difference(states)
inconsistent_categories

In [ ]:
# Metoda .isin zwraca wszystkie wiersze z ramki danych, w których został spełniony p[rzekazany warunek]
inconsistent_rows = flights_df["ORIGINSTATENAME"].isin(inconsistent_categories)
flights_df[inconsistent_rows]

In [ ]:
# Sprytna sztuczka, której możemy użyć do usunięcia wierszy
# Jak myślisz, co oznacza ~?
flights_df = flights_df[~inconsistent_rows]

In [ ]:
set(flights_df['ORIGINSTATENAME'])

### Zmiana mapowania kategorii

To, co widzieliśmy powyżej, to dane, których nie było w predefiniowanym zestawie kategorii. Jednak możemy również natknąć się na inny rodzaj problemów z danymi kategorycznymi, które lepiej rozwiązywać poprzez ponowne mapowanie kategorii niż usuwanie danych. Odpowiednie miejsca do wykonania tego ponownego mapowania to:
1. **Niespójność wartości**: „żonaty”, „niezamężna”, „stanu wolnego”, „ożeniony” <br>
 1. Uważaj też na końcowe białe znaki!
2. **Konwertowanie danych na kategorie lub zbyt wiele kategorii**: Załóżmy, że w naszej ramce danych mamy kolumnę dochodu gospodarstwa domowego.
 1. Możemy zmienić ten typ danych na kategoryczny, grupując dochody (np. `0 - 20k`, `20k - 40k`, `40k - 60k`, `60k +` itd.).
 2. Możemy również zredukować to dalej do `low_class`, `middle_class`, `upper_class`

Zajmijmy się nimi w kolejności. W naszej ramce danych lotów kolumny „CANCELLED” i „DIVERTED” przyjmują niespójne wartości. Być może najbezpieczniejszą opcją jest uruchomienie `.value_counts()` na jednej z tych kolumn (`.value_counts()` działa na danych typu `Series`)

In [ ]:
flights_df["CANCELLED"].value_counts()

Świetnie! Widzimy więc, że nasze Fałszywe wartości mogą przyjąć jedną z trzech wartości, a wartości Prawdy też są podobne. Możemy arbitralnie wykorzystać te, których chcemy użyć, posuwając się do przodu. Dla jednoznaczności wybierzmy odpowiednio False i True.

In [ ]:
## Skorzystaj z metody replace: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.replace.html
## Aby zamienić 0, F, 1 and T na odpowiadające wartości w kolumnie CANCELLED
flights_df["CANCELLED"].replace(["0", "F"], "False", inplace=True)
flights_df["CANCELLED"].replace(["1", "T"], "True", inplace=True)
flights_df["CANCELLED"].value_counts()

In [ ]:
flights_df["DIVERTED"].value_counts()

In [ ]:
# Możemy alternatywnie użyć słownika, aby "zredukować" nasze kategorie.
mapping = {"F": "False", "0": "False", "1": "True", "T": "True"}
flights_df["DIVERTED"] = flights_df["DIVERTED"].replace(mapping)
flights_df["DIVERTED"].value_counts()

Jak wspomniano wcześniej, inną sytuacją, w której możemy chcieć ponownie przyporządkować kategorie, jest zmniejszenie liczby wartości w kolumnie. W naszym przypadku załóżmy, że firma lotnicza chciałaby sklasyfikować loty na podstawie długości tras. Tak więc wszystko między 0 a 1000 mil jest `short`, między 1000 a 2500 to `medium`, a 2500+ to `long`.

Możemy użyć metody [`.cut`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html) aby zaklasyfikować dane. Musimy podać 3 argumenty:
1. `Series`, którą chcemy sklasyfikować
2. Bins - liczbę przedziałów
3. Labels - etykiety do przypisania przydziałom

In [ ]:
import numpy as np

bins = [0, 1000, 2500, np.inf]
labels = ["short", "medium", "long"]
flights_df["DISTANCE_CATEGORY"] = pd.cut(flights_df["DISTANCE"], bins=bins, labels=labels)

flights_df[["DISTANCE", "DISTANCE_CATEGORY"]]

In [ ]:
flights_df[flights_df["DISTANCE_CATEGORY"] == "long"]

### Radzenie sobie z danymi datoczasowymi

Jednym z typowych problemów, z którymi się spotkasz, jest zajmowanie się datami i godzinami. Czemu? Ponieważ istnieje wiele sposobów formatowania daty, na przykład `DD/MM/RRRR`, `MM/DD/RR`, `X. MIESIĄC ROK` itp. W powyżej ramce danych nasze daty są w rzeczywistości sformatowane jako jedna numer. Pandas dostarcza nam przydatnego pomocnika do konstruowania dat i godzin — to znaczy metodę [`.to_datetime()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html ).

Zanim się tym zajmiemy, warto szybko przedstawić, w jaki sposób daty są zwykle przechowywane na komputerach. Zazwyczaj daty są obliczane na podstawie liczby sekund, które upłynęły od **1 stycznia 1970**. Gdy chcemy, znaleźć różnicę w czasie między 3.02.2013 16:00 a 21.01.2013 09:00, program wykonuje swoje operacje na **czasie Epoka/Unix/POSIX** dla tych wartości, a następnie możemy coś zakodować, aby otrzymać wartość z powrotem w wybranym przez nas formacie (np. 13 dni, 7 godzin). Posługując się liczbami:

- **3/2/2013 16:00** = 1,359,907,200
- **21/1/2013 09:00** = 1,358,758,800

Różnica w datach = 1,359,907,200 - 1,358,758,800 = 1,148,400 sekund

`format(1148400) = 13 dni, 7 godzin`

Przykłady formatowania dat:
<table>
    <tr>
        <td><b>Date</b></td>
        <td><b>Datetime format</b></td>
    </tr>
    <tr>
        <td>15th June 2020</td>
        <td>%c</td>
    </tr>
    <tr>
        <td>15/06/2020</td>
        <td>%d/%m/%Y</td>
    </tr>
    <tr>
        <td>06-15-2020</td>
        <td>%m-%d-%Y</td>
    </tr>
</table>


In [ ]:
pd.to_datetime(flights_df["FLIGHTDATE"])

To dlaczego teraz wszystkie daty to 1970-01-01?

A to dlatego, że daty są wewnętrznie przechowywane jako sekundy (liczby). Nasza kolumna `FLIGHTDATE` również wyświetla daty lotów w postaci liczb. Tak więc, kiedy uruchamiamy metodę `.to_datetime()`, wszystkie nasze daty są interpretowane jako czas POSIX.

Jednym z prostych rozwiązań, aby to naprawić, jest jawne określenie naszego formatu daty i godziny. Biorąc pod uwagę powyższe przykłady, jak myślisz, jaki będzie format daty?
One simple solution we can do to fix that is to explicitly specify our datetime?

In [ ]:
## Przypisywanie formatu
date_format = "%Y%m%d"
flights_df["FLIGHTDATE"] = pd.to_datetime(flights_df["FLIGHTDATE"], format=date_format)
flights_df["FLIGHTDATE"]

To rozwiązanie było dość specyficzne dla problemu, który mieliśmy pod ręką. Ale w prawdziwym świecie często można napotkać mieszane formaty dat w jednej ramce danych. Na przykład:

<table>
    <tr>
        <td><b>Name</b></td>
        <td><b>Date of Birth</b></td>
        <td><b>Age</b></td>
    </tr>
    <tr>
        <td>John</td>
        <td>01/07/1995</td>
        <td>25</td>
    </tr>
    <tr>
        <td>Jane</td>
        <td>20-04-1992</td>
        <td>28</td>
    </tr>
    <tr>
        <td>Mark</td>
        <td>3rd January 1990</td>
        <td>30</td>
    </tr>
    </table>


`.to_datetime()` ponownie przychodzi tutaj na ratunek! W poprzedniej komórce kodu wyraźnie ustawiliśmy format daty (ze względu na nietypowy charakter sposobu przechowywania tej daty w ramce danych) - ale bardziej ogólnie możemy użyć `.to_datetime()`, aby automatycznie wywnioskować format każdej daty z osobna.

```python
# errors='coerce' means we'll return NA rows for invalid dates
df["DATE"] = pd.to_datetime(df["DATE"], infer_datetime_format=True, errors='coerce')
```

## Walidacja krzyżowa

Co to znaczy sprawdzić integralność naszych danych? Zasadniczo musimy mieć świadomość, że kolumna danych, które widzimy, jest spójna w oparciu o inne kolumny danych. Właśnie to sprawdza się w **walidacji krzyżowej**. Zanim rozszerzę niektóre walidacje krzyżowe w tym zbiorze danych, przedstawię nieco bardziej trywialny przykład, aby pokazać, gdzie niewykonanie takich kontroli może zniekształcić analizę:

Poniższa fikcyjna tabela pokazuje wpisy niektórych posiadaczy kredytów studenckich na studia licencjackich (UG) i podyplomowych (PG). Zestaw danych składa się z imienia i nazwiska kredytobiorcy, daty urodzenia (DOB), obecnego wieku (lub wieku zmarłego, jeśli dotyczy, niezależnie od tego, czy zmarli, czy też nie), kwoty ich kredytu UG i PG oraz całkowite kwoty, którą są winni – co powinno być sumą dwóch poprzednich pól. W poniższej tabeli zaznaczono wątpliwe pola kursywą.

<table>
    <tr>
        <td><b>Name</b></td>
        <td><b>D.O.B</b></td>
        <td><b>Age</b></td>
        <td><b>Deceased</b></td>
        <td><b>U.G Loan (£)</b></td>
        <td><b>P.G Loan (£)</b></td>
        <td><b>Total Loan (£)</b></td>
    </tr>
    <tr>
        <td>Idaline</td>
        <td>1971-04-27</td>
        <td>49</td>
        <td>F</td>
        <td>24100</td>
        <td>11900</td>
        <td>36000</td>
    </tr>
    <tr>
        <td>Freddie</td>
        <td>1962-12-27</td>
        <td>57</td>
        <td>F</td>
        <td>26600</td>
        <td>12600</td>
        <td>39200</td>
    </tr>
    <tr>
        <td>Debee</td>
        <td>1970-11-19</td>
        <td>49</td>
        <td>F</td>
        <td>32400</td>
        <td>97000</td>
        <td><i>42100</i></td>
    </tr>
    <tr>
        <td>Joyann</td>
        <td>1957-01-24</td>
        <td><i>41</i></td>
        <td>T</td>
        <td>24400</td>
        <td>11500</td>
        <td>35900</td>
    </tr>
    <tr>
        <td>Ajay</td>
        <td>1960-05-12</td>
        <td><i>50</i></td>
        <td>F</td>
        <td>25500</td>
        <td>18800</td>
        <td>44300</td>
    </tr>
    <tr>
        <td>Emelia</td>
        <td>1957-11-23</td>
        <td><i>57</i></td>
        <td>T</td>
        <td>34000</td>
        <td>17500</td>
        <td><i>0</i></td>
    </tr>
            
</table>
            
            ajay, emelia, joyann

In [ ]:
html_table = """
<table>
    <tr>
        <td><b>Name</b></td>
        <td><b>D.O.B</b></td>
        <td><b>Age</b></td>
        <td><b>Deceased</b></td>
        <td><b>U.G Loan (£)</b></td>
        <td><b>P.G Loan (£)</b></td>
        <td><b>Total Loan (£)</b></td>
    </tr>
    <tr>
        <td>Idaline</td>
        <td>19710427</td>
        <td>50</td>
        <td>F</td>
        <td>24100</td>
        <td>11900</td>
        <td>36000</td>
    </tr>
    <tr>
        <td>Freddie</td>
        <td>19621227</td>
        <td>58</td>
        <td>F</td>
        <td>26600</td>
        <td>12600</td>
        <td>39200</td>
    </tr>
    <tr>
        <td>Debee</td>
        <td>19701119</td>
        <td>49</td>
        <td>F</td>
        <td>32400</td>
        <td>97000</td>
        <td><i>42100</i></td>
    </tr>
    <tr>
        <td>Joyann</td>
        <td>19570124</td>
        <td><i>41</i></td>
        <td>T</td>
        <td>24400</td>
        <td>11500</td>
        <td>35900</td>
    </tr>
    <tr>
        <td>Ajay</td>
        <td>19600512</td>
        <td><i>50</i></td>
        <td>F</td>
        <td>25500</td>
        <td>18800</td>
        <td>44300</td>
    </tr>
    <tr>
        <td>Emelia</td>
        <td>19571123</td>
        <td><i>57</i></td>
        <td>T</td>
        <td>34000</td>
        <td>17500</td>
        <td><i>0</i></td>
    </tr>

</table>
"""

html_df = pd.read_html(html_table, header=0)[0]
html_df

Najpierw popracujmy nad zmienną **Age**. Zgodnie z naszą dokumentacją danych, wiek w komórce powinien odzwierciedlać aktualny wiek kredytobiorców. Wyjątkiem jest sytuacja, gdy kredytobiorca nie żyje, w którym to przypadku wiek powinien zawierać wiek kredytobiorcy w momencie jego śmierci. Najpierw ustalmy, które wiersze łamią ten warunek.

In [ ]:
# najpierw zmieńmy nazwy niektórych kolumn
html_df.columns = ["name", "dob", "age", "deceased", "ug_loan", "pg_loan", "total_loan"]
html_df.info()

In [ ]:
## Zamie nmy 'dob' na obiekt date
html_df["dob"] = pd.to_datetime(html_df["dob"], format="%Y%m%d")
html_df

In [ ]:
# Stwórz nową kolumnę „now_date” wypełnioną aktualną datą i godziną
html_df["now_date"] = pd.Timestamp(pd.datetime.now())

## Oblicz różnicę między 'dob' i 'now_date' i zwróć wartość jako lata
now_date_dob_difference = html_df["now_date"] - html_df["dob"]
now_date_dob_difference

In [ ]:
# Ta linia zmienia obiekty timedate na rok zmiennoprzecinkowy, który następnie konwertujemy na int
now_date_dob_difference = (now_date_dob_difference / np.timedelta64(1, 'Y')).astype("int64")
now_date_dob_difference

In [ ]:
# Na oko możemy zobaczyć, który wiek nie pasuje do ramki danych, którą pokazaliśmy wcześniej.
# Jednak ogólnie zakodujmy to za pomocą logiki pandas.
## Zwróć wiersze, w których „now_date_dob_difference” różni się od zmiennej wieku ramki danych
html_df[html_df["age"] != now_date_dob_difference]

Przyjrzyjmy się, dlaczego powyższe komórki zostały zwrócone. Jak wspomniano wcześniej, jeśli pożyczkobiorca nie żyje, jego wiek powinien to odzwierciedlać. Oznacza to, że wiek Joyann i Emelii jest rzeczywiście prawidłowy. Używając logiki, odfiltrujmy te wiersze, aby zwrócić tylko te wiersze, które mają matematycznie niepoprawny wiek.

In [ ]:
## Odfiltruj odpowiednich pożyczkodawców za pomocą logiki (wskazówka: &)
incorrect_age_rows = html_df[(html_df["age"] != now_date_dob_difference) & (html_df["deceased"] == "F")]
incorrect_age_rows

In [ ]:
## Zaktualizuj ramkę danych invalid_age_rows o poprawiony wiek
incorrect_age_rows["age"] = ((incorrect_age_rows["now_date"] - incorrect_age_rows["dob"]) / np.timedelta64(1, 'Y')).astype("int64")
incorrect_age_rows

In [ ]:
## Teraz zaktualizuj odpowiednie wpisy html_df o kolumnę wieku z ramki danych invalid_age_rows
html_df.update(incorrect_age_rows["age"])
html_df

In [ ]:
## Konwertuj wiek z powrotem na int
html_df["age"] = html_df["age"].astype("int64")
## Usuń kolumnę now_date
html_df.drop("now_date", axis=1, inplace=True)
html_df

Popracujmy teraz nad kwotami pożyczki. Zwróć wszystkie kolumny, w których `ug_loan` + `pg_loan` nie są równe `total_loan`

In [ ]:
## Podzbiór `ug_loan` i `pg_loan` z naszej ramki danych, a następnie sumowanie wzdłuż osi kolumny
sum_loans = html_df[["ug_loan", "pg_loan"]].sum(axis=1)
sum_loans

In [ ]:
## Zwróć wiersze, które mają nieprawidłowe wartości sum
incorrect_loan_rows = html_df[html_df["total_loan"] != sum_loans]
incorrect_loan_rows

### Jak postępować z polami, które nie przeszły walidacji?

Tutaj widzimy dwa wiersze, które nie zawierają poprawnych całkowitych kwot kredytu. Analizując każdy z nich z osobna, widzimy, że dane w pierwszym wierszu najprawdopodobniej zawierały niepoprawną wartość `pg_loan` (97 000 GBP w przypadku pożyczki podyplomowej). W drugim, z jakiegoś powodu wartość `total_loan` nie została obliczona. Naiwną strategią może być nadpisanie całkowitych kwot pożyczki sumą `ug_loan` i `pg_loan`. To naprawia typy błędów, w których zwracany jest drugi wiersz. Jednak może istnieć podstawowy problem z powodu pierwszego rzędu. Jeśli zsumujemy tutaj `ug_loan` i `pg_loan`, utworzymy **obserwację odstającą**. W prawdziwym zbiorze danych mogą wystąpić bardzo realne zagrożenia, takie jak te, które mogą narazić na szwank integralność danych – takie problemy mogą łatwo wymykać się spod kontroli, więc upewnij się, że poświęcisz czas na przemyślenie, w jaki sposób Twoje działania wpłyną na Twój dane.

Jak wspomniano wcześniej, niektóre aspekty nauki o danych są sztuką – ale jakąkolwiek decyzję heurystyczną podejmiemy, musimy znaleźć dla niej mocne uzasadnienie. W tym konkretnym przypadku zamierzam usunąć wiersze z niepoprawnym `total_loan`, ponieważ ten błąd prawdopodobnie wystąpił z powodu błędu we wprowadzaniu danych przez człowieka. Wiersze z `total_loan = 0` prawdopodobnie wystąpiły z powodu jakiegoś systematycznego błędu - być może z innej bazy danych, w której nie podano sumy total_loan. Biorąc pod uwagę inne weryfikacje, jednym z rozwiązań, które moglibyśmy wybrać, jest zsumowanie dwóch kolumn.

In [ ]:
## Zidentyfikuj wiersze, w których total_loan NIE jest równe 0, ale jest niepoprawne
incorrect_loan_but_not_zero_rows = html_df[(html_df["total_loan"] != sum_loans) & (html_df["total_loan"] != 0)]
print(incorrect_loan_but_not_zero_rows)

## Usuń te wiersze
html_df = html_df.drop(incorrect_loan_but_not_zero_rows.index)

html_df

In [ ]:
html_df[(html_df["total_loan"] == 0)]


In [ ]:
html_df.drop(index=5, inplace=True)

In [ ]:
# Zakładając, że jesteśmy zadowoleni ze wszystkich innych wpisów w naszych pożyczkobiorcach, możemy bezpośrednio obliczyć i nadpisać total_loan w naszej ramce danych
## Zastąp total_loan sumą ug_loan i pg_loan
html_df["total_loan"] = html_df["ug_loan"] + html_df["pg_loan"]
html_df

## Praca z danymi tekstowymi i typu string

Dane tekstowe są oczywiście niezwykle powszechnym rodzajem danych i mogą przybierać różne formy – od tekstu nieustrukturyzowanego po e-maile, nazwiska, numery telefonów itp. Istnieje wiele rodzajów problemów, które możemy napotkać w przypadku danych tekstowych:
- Niespójność danych (np. +86 195 448 8582 vs 0086-195-448-8582)
- Naruszenia tekstu (np. niedozwolone znaki, błędy w polach wejściowych, literówki w tekście)
- Literówki „strukturalne” (np. +86.1954.48858.2)

W przykładowej tabeli poniżej widzimy listę osób wraz z ich imionami i numerami telefonów. Jak widać – najprawdopodobniej ze względu na wolne pola tekstowe, nazwiska i numery telefonów zostały wprowadzone w różnych formatach. Naszym zadaniem jest standaryzacja tych pól, aby były spójne w całej ramce danych:

<table>
    <tr>
        <td><b>Name</b></td>
        <td><b>Phone Number</b></td>
    </tr>
    <tr>
        <td>Dr Darci Abela</td>
        <td>+86-185-338-1819</td>
    </tr>
    <tr>
        <td>Mr Patten St. Queintain</td>
        <td>00865872411917</td>
    </tr>
    <tr>
        <td>mr conant burden</td>
        <td>0086-289-702-0948</td>
    </tr>
    <tr>
        <td>miss marcia Dutnell</td>
        <td>0668</td>
    </tr>
    <tr>
        <td>dr Greggory lurner</td>
        <td>+31 778 813 8432</td>
    </tr>
    <tr>
        <td>MS Doe Beavan</td>
        <td>+420-731-276-7633</td>
    </tr>
    <tr>
        <td>Tamarah Delgado</td>
        <td>+868431029051</td>
    </tr>
    <tr>
        <td>Miss Arlee daborne</td>
        <td>+33-307-220-2746</td>
    </tr>
    <tr>
        <td>Ly b. Grima</td>
        <td>+238-863-946-4232</td>
    </tr>
</table>

Użyjemy małego sztucznie stworzonego zbioru danych w csv w tym celu.

In [ ]:
# np = names_phones
np_df = pd.read_csv(f"{path}mock_names_phones.csv", header=0, index_col=0)
np_df

Ok - dla tego dataframe są 4 zadania:
1. Utwórz kolumnę title, która zawiera tytuł każdej osoby (np. Pani, Panna itp.). Ta kolumna powinna być znormalizowana i kategoryczna
2. Podziel kolumnę name na kolumnę first name i last name. Obie kolumny powinny mieć pierwszą literę imienia z wielkiej litery
3. Usuń wiersz `name`
4. Standaryzuj numery telefonów w formacie `00XXXXXXXXX`. To znaczy - dwa zera poprzedzone resztą rzeczywistej liczby

Zajmijmy się nimi w kolejności

In [ ]:
# Najpierw chcemy utworzyć nową kolumnę tytułową, która przyjmuje tytuły grzecznościowe w kolumnie imienia
# Aby to uzyskać, musimy podzielić name po białym znaku i wziąć pierwszy element z listy podzielonej
example_string = "this string will be split"
print(example_string.split())
print(example_string.split()[0])

In [ ]:
# Aby wykonać operacje na stringach naa kolumnach stringów w pandas, musimy poprzedzić naszą funkcję ciągów znakiem „.str”
np_df["name"].str.split()


In [ ]:
def capitalize(txt):
  return txt.capitalize()

In [ ]:
## Utwórz i wypełnij kolumnę title.
# To zadanie można rozwiązać na kilka różnych sposobów.
# Zobacz, ile rozwiązań możesz wymyślić
np_df["title"] = np_df["name"].str.split().apply(lambda x: x[0])
np_df

In [ ]:
# Chcemy, aby nasz title był ustandaryzowany i kategoryczny.
## Zamień kolumnę na kolumnę kategoryczną i zwróć wszystkie kategorie, które obecnie istnieją w kolumnie
np_df["title"] = np_df["title"].astype("category")
set(np_df["title"])

In [ ]:
# Widzimy wiele różnych wariantów. Wybierzmy metodę normalizacji wpisów (np. wielkie litery).
## Standaryzuj kolumnę title
np_df["title"] = np_df["title"].str.upper()
np_df

In [ ]:
## W podobny sposób do powyższego utwórz nową kolumnę na imię i jedną na nazwisko.
# Upewnij się, że dla obu nowych kolumn nazwy są pisane małymi literami, z wyjątkiem pierwszej litery, która jest pisana wielką literą
np_df["first_name"] = np_df["name"].str.split().apply(lambda x: x[1].lower())
np_df["last_name"] = np_df["name"].str.split().apply(lambda x: x[-1].lower())
np_df["first_name"] = np_df["first_name"].str[0].str.upper() + np_df["first_name"].str[1:]
np_df["last_name"] = np_df["last_name"].str[0].str.upper() + np_df["last_name"].str[1:]

np_df

In [ ]:
## Usuń kolumnę name
np_df.drop("name", axis=1, inplace=True)
np_df

Świetnie! To prowadzi nas do czwartej części zadania - ujednolicenia numeru telefonu i przekonwertowania go na typ danych int. Przypomnij sobie, jak chcemy, aby nasze numery telefonów wyglądały: zacznij od 00, a następnie do reszty numeru.

In [ ]:
# Zwraca wszystkie (unikalne) numery telefonów, dzięki czemu możemy zobaczyć różne typy problemów jakie nas czekają
set(np_df["phone number"])

Ok, więc jakie problemy widzisz?
<details>
     <summary><b>> Kliknij tutaj, aby zobaczyć problemy</b></summary>
     <ul>
         <li>Numery zaczynają się różnie – niektóre zaczynają się od `+`, inne od `00`</li>
         <li>Niektóre numery mają spacje między grupami liczb, inne są dzielone. Niektóre numery również nie mają „grup”</li>
         <li>Niektóre numery zaczynają się od spacji, inne od `+ `, inne od `+`.</li>
         <li>Niektóre numery mają tylko cztery liczby</li>
     </ul>
</details>

Istnieje kilka sposobów formatowania tych ciągów do pożądanego wyniku. Tutaj poprowadzę cię przez metodę, w której iterujemy wiersze i stosujemy funkcję, aby ponownie przypisać zmienną. Zacznijmy od stworzenia funkcji pośredniej, która pobiera numer telefonu i manipuluje nim do pożądanego wyniku.

In [ ]:
def standardise_phone_number(phone_number):

    ## jeśli pierwszym znakiem jest "+", usuń go.
    if phone_number.startswith("+"):
        phone_number = phone_number.replace("+", "")

    ## usuń wszystkie spacje z numeru telefonu
    phone_number = phone_number.replace(" ", "")

    ## usuń myślniki z numeru telefonu
    phone_number = phone_number.replace("-", "")

    ## jeśli numer nie zaczyna się od 00, dodaj 00 do początku numeru
    if not phone_number.startswith("00"):
        phone_number = "00" + phone_number

    ## zwróć numer telefonu
    return phone_number

In [ ]:
# Będziemy iterować po wierszach ramki danych i ponownie przypiszemy wiersz do standardowego wariantu
for index, row in np_df.iterrows():

    ## Odwołaj się do naszej funkcji standaryzacji na numer telefonu dla bieżącej pętli
    row["phone number"] = standardise_phone_number(row["phone number"])

np_df

In [ ]:
# W naszej ramce danych nadal znajdują się nieprawidłowe liczby (tj. te, które pierwotnie miały długość 4)
## Zamień wszystkie numery telefonów poniżej 10 cyfr/znaków na pd.NA
# Podpowiedź: będzie potrzebna metoda .loc
np_df.loc[np_df["phone number"].str.len() < 10, "phone number"] = pd.NA
np_df

In [ ]:
# Skoncentrujemy się na brakujących danych w następnej części, ale policzmy liczbę wierszy z NA i usuńmy je
null_phone_numbers = np_df["phone number"].isnull()
print("Number of null phone numbers:", null_phone_numbers.sum())

# Usuń wiersze, które mają puste numery telefonów
np_df = np_df.dropna(subset=["phone number"])
np_df

Bardziej skomplikowane manipulacje ciągami znaków można wykonać za pomocą **wyrażeń regularnych**, znanych również jako [regex](https://docs.python.org/3/howto/regex.html). Nie będziemy tutaj przyglądać się wyrażeniu regularnemu, ale ważne jest, aby wiedzieć o jego mocy. Zasadniczo wyrażenie regularne pozwala nam określić reguły dla ciągów, które chcemy dopasować. Ma bardzo szerokie zastosowanie. Oto kilka przykładów:
- identyfikuj wiadomości e-mail w zakresach tekstu
- sprawdź, czy adres URL ma poprawny format
- wyodrębnij tylko cyfry z ciągu tekstowego

Kiedy natkniesz się na zadania, które wymagają oczyszczenia danych tekstowych, regex jest narzędziem do tego zadania.

## Scalanie danych razem (merging)

Czasami znajdziemy dane w różnych plikach, które musimy połączyć w jedną ramkę danych, zanim będziemy mogli jej użyć. Przykładem tego są [dane IMDB](http://www.imdb.com/interfaces#plain) – gdzie, być może ze względu na rozmiar dostępnych danych, twórcy postanowili podzielić cały zbiór danych na wiele osobnych plików. Przyjrzyjmy się szybko, jak możemy połączyć dane z tych wielu plików.

Źródło:
IMDb
(http://www.imdb.com).

Pliki do pobrania i scalenia to:
- tytuł.akas.tsv.gz
- tytuł.podstawy.tsv.gz
- tytuł.oceny.tsv.gz

In [ ]:
# URUCHOM TĘ KOMÓRKĘ TYLKO JEŚLI CHCESZ PONOWNIE POBRAĆ CAŁE DANE
import requests
import gzip
import pandas as pd
import pickle
import os

DATA_FOLDER = "DATA" # TODO: najpierw ten folder trzeba utworzyc
DOWNLOAD_URL = "https://datasets.imdbws.com/"
files_to_download = ["title.ratings.tsv.gz", "title.akas.tsv.gz", "title.basics.tsv.gz"]


for file_string in files_to_download:
    # pobierz plik tsv.gz
    # plik jest obiektem request
    print("Getting request for file:", file_string)
    df = pd.read_csv(DOWNLOAD_URL + file_string, sep="\t", compression="gzip")
    df = df.iloc[-10000:]
    print(df.shape)
    pickle.dump(df, open(os.path.join(DATA_FOLDER, file_string.replace(".tsv.gz", ".df")), "wb"))
    print(df.head())


In [ ]:
ratings_df = pickle.load(open(os.path.join(DATA_FOLDER, "title.ratings.df"), "rb"))
films_df = pickle.load(open(os.path.join(DATA_FOLDER, "title.akas.df"), "rb"))
basics_df = pickle.load(open(os.path.join(DATA_FOLDER, "title.basics.df"), "rb"))

### Łączenie wielu ramek danych

Podobnie jak w SQL, chcemy wykonać jakiś rodzaj [`.join()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html) lub [ `.merge()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html#pandas.DataFrame.merge) przez ramki danych, które obecnie posiadamy. Wymaga to ustawienia unikalnego identyfikatora/klucza podstawowego jako indeksu ramek danych. Pamiętaj o dwóch głównych typach złączeń,  **inner** i **lefyt**. Jak myślisz, które podejście będzie bardziej poprawne w tym przypadku?

In [ ]:
## zmodyfikuj indeksy ratings_df, basics_df i movies_df, aby wszystkie miały wspólny indeks
ratings_df = ratings_df.set_index("tconst")
basics_df = basics_df.set_index("tconst")
films_df = films_df.set_index("titleId")
films_df.head()

In [ ]:
## Wykonaj join lub merge z basics_df i ratings_df za pomocą movies_df
df = films_df.join([basics_df, ratings_df])
df

---